<a href="https://colab.research.google.com/github/doowilliams/Image-Classification-Using-HOG-Features-and-KNN/blob/main/williams2CWK50_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile


# Unzip our data
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/dataset/cat_dog1.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [3]:
# Setup the image directories
image_dir = "/content/cat_dog1"


In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

def load_images_and_labels(image_dir, limit=None):
    images = []
    labels = []
    count = 0
    for filename in os.listdir(image_dir):
        if limit is not None and count >= limit:
            break
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)
            labels.append(1 if filename.startswith('dog') else 0)  # Assuming dog filenames start with 'dog'
            count += 1
    return images, labels

def bag_of_visual_words(images, n_clusters=100):
    sift = cv2.SIFT_create()
    descriptors = []
    for img in images:
        kp, des = sift.detectAndCompute(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), None)
        if des is not None:
            descriptors.append(des)

    descriptors = np.vstack(descriptors)
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=100, max_iter=100)
    kmeans.fit(descriptors)

    histograms = []
    for des in descriptors:
        histogram, _ = np.histogram(kmeans.predict(des.reshape(1, -1)), bins=range(n_clusters+1))
        histograms.append(histogram)
    histograms = np.vstack(histograms)

    combined_histogram = np.sum(histograms, axis=0)  # Combine histograms along the rows

    scaler = StandardScaler()
    normalized_histogram = scaler.fit_transform(combined_histogram.reshape(1, -1))  # Reshape for normalization

    return normalized_histogram

image_dir = "/content/cat_dog1"
images, labels = load_images_and_labels(image_dir)

# Feature extraction and histogram creation
histograms = bag_of_visual_words(images)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(histograms, labels, test_size=0.2, random_state=42)

# Train classifier
classifier = KNeighborsClassifier(n_neighbors=40)
classifier.fit(X_train, y_train)

# Predict
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred)
print("Accuracy (KNN):", accuracy_knn)


KeyboardInterrupt: 

In [ ]:
import os
import cv2
import numpy as np
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


# Function to load images
def loadImages(path, limit=10000):
    images = []
    classes = []
    count = 0
    for filename in os.listdir(path):
        if count >= limit:
            break
        category = filename.split('.')[0]
        cls = 1 if category == 'dog' else 0
        img_path = os.path.join(path, filename)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (128, 128))
        images.append(image)
        classes.append(cls)
        count += 1
    return np.array(images), np.array(classes)

# Load images and labels
images, classes = loadImages("/content/cat_dog1")
X_train, X_test, y_train, y_test = train_test_split(images, classes, test_size=0.2, random_state=42)

# Function to extract HOG features
def hogFeatures(image):
    features = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
    return features

hog_features = []
for image in X_train:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_feature = hogFeatures(gray_image)
    hog_features.append(hog_feature)

# K-means clustering on HOG features to generate visual words vocabulary
kmeans = MiniBatchKMeans(n_clusters=100, random_state=42)
kmeans.fit(hog_features)
centroids = kmeans.cluster_centers_


# Reduce dimensions using PCA
pca = PCA(n_components=2)
hog_features_pca = pca.fit_transform(hog_features)
cluster_labels = kmeans.predict(hog_features)

# Plotting the clusters
plt.scatter(hog_features_pca[:, 0], hog_features_pca[:, 1], c=cluster_labels, cmap='viridis', marker='.', label='Data Points')
plt.title('K-means clustering with PCA')
plt.show()

# BOW representation
x_train_bovw = np.array([np.histogram(kmeans.predict([hogFeatures(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))]), bins=range(101))[0] for image in X_train])
x_test_bovw = np.array([np.histogram(kmeans.predict([hogFeatures(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))]), bins=range(101))[0] for image in X_test])

# Train KNN classifier
classifier = KNeighborsClassifier(n_neighbors=40)
classifier.fit(x_train_bovw, y_train)

# Predictions
y_pred = classifier.predict(x_test_bovw)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred)
print("Accuracy (KNN):", accuracy_knn)


In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler

# Implementation of Bag of Visual Words-based Approach
def bag_of_visual_words(image_dir, n_clusters=50):
    # Read images from the directory
    images = []
    for filename in os.listdir(image_dir):
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)

    # Convert images to grayscale
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]

    # Placeholder for feature extraction
    sift = cv2.SIFT_create()
    keypoints = []
    descriptors = []
    for img in gray_images:
        kp, des = sift.detectAndCompute(img, None)
        keypoints.append(kp)
        descriptors.append(des)

    # Stack descriptors
    descriptors = np.vstack(descriptors)

    # Cluster the features to create visual words
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=100, max_iter=100)
    kmeans.fit(descriptors)
    visual_words = kmeans.cluster_centers_

    # Build histograms of visual words for each image
    histograms = []
    for des in descriptors:
        histogram, _ = np.histogram(kmeans.predict(des.reshape(1, -1)), bins=range(n_clusters+1))
        histograms.append(histogram)
    histograms = np.vstack(histograms)

    # Normalize the histograms
    scaler = StandardScaler()
    normalized_histograms = scaler.fit_transform(histograms)

    return normalized_histograms


In [ ]:
histograms = bag_of_visual_words(image_dir)
print("Histograms shape:", histograms.shape)

In [ ]:
from sklearn.neighbors import NearestNeighbors


# Function to resize images
def resize_image(img, target_size=(128, 128)):
    return cv2.resize(img, target_size)

# Implementation of Bag of Visual Words-based Approach
def bag_of_visual_words(image_dir, n_clusters=50):
    # Read images from the directory
    images = []
    for filename in os.listdir(image_dir):
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images.append(img)

    # Convert images to grayscale and resize them
    gray_images = [resize_image(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)) for img in images]

    # Placeholder for HOG feature extraction
    hog = cv2.HOGDescriptor()

    # Extract HOG features
    features = []
    for img in gray_images:
        try:
            feature = hog.compute(img)
            if feature is not None and len(feature) > 0:
                features.append(feature.flatten())
        except Exception as e:
            print(f"Error processing image: {e}")

    if len(features) == 0:
        raise ValueError("No valid HOG features found.")

    features = np.array(features)

    # Cluster the features to create visual words
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=100, max_iter=100)
    kmeans.fit(features)
    visual_words = kmeans.cluster_centers_

    # Build histograms of visual words for each image
    histograms = []
    for feature in features:
        histogram, _ = np.histogram(kmeans.predict(feature.reshape(1, -1)), bins=range(n_clusters+1))
        histograms.append(histogram)
    histograms = np.vstack(histograms)

    # Normalize the histograms
    scaler = StandardScaler()
    normalized_histograms = scaler.fit_transform(histograms)

    return normalized_histograms

# Call the function
try:
    histograms = bag_of_visual_words(image_dir)
    print("Histograms shape:", histograms.shape)
except ValueError as ve:
    print(ve)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

histograms = bag_of_visual_words(image_dir)
print("Histograms shape:", histograms.shape)

# code for k-NN classification
def knn_classification(X_train, y_train, X_test, k=5):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    return knn.predict(X_test)

# K-NN usage
X_train = histograms
y_train = np.random.randint(2, size=len(os.listdir(image_dir)))  # Random labels for demonstration
X_test = histograms[:10]  # Example test data
predictions = knn_classification(X_train, y_train, X_test)
print("Predictions:", predictions)

 # Report: Critique and Recommendation for Petfinder.com's Image Classifier

## Introductiuon:

Petfinder.com aims to develop an efficient binary image classifier capable of distinguishing between images of cats and dogs. The proposed solution involves a supervised learning approach using bags of visual words with k-means clustering of HOG features as features and k-nearest neighbors (KNN) as the classifier. This report critically evaluates this proposal in light of the stakeholders' wishes and makes a final recommendation.

## Stakeholders' Wishes:

1. **Lead Data Scientist (S1)**:
* Requirement: The system should handle wide
variations in imaging conditions.
* Analysis: The use of bags of visual words with HOG features may not fully address this requirement. While HOG features are robust to some variations, they may not handle all types of variations efficiently.
2. **Lead Software Engineer (S2)**:
* Requirement: The system should be implementable in-house with no licensing issues.
* Analysis: The proposed solution satisfies this requirement as it uses open-source libraries like OpenCV and scikit-learn.
3. **Head of IT (S3)**:
* Requirement: The system should be computationally efficient.
* Analysis: The proposed solution may have high computational overhead during both training and prediction phases due to feature extraction, clustering, and KNN classification.
4. **Lead Data Engineer (S4)**:
* Requirement: The online content moderators should have a role after deployment.
* Analysis: The proposed solution does not directly address this requirement. Additional features might need to be incorporated to involve online content moderators.
5. **Charitable Donor (S5)**:
* Requirement: The system should be interpretable.
* Analysis: The proposed solution lacks interpretability. KNN as a classifier does not offer insight into decision-making, and bags of visual words may not be easily interpretable.

# Critical Analysis

* **Features**: The use of bags of visual words with HOG features may not adequately capture all variations in imaging conditions. While HOG features are robust to some extent, they may not handle variations like scale, rotation, and backgrounds effectively.
* **Classifier**: K-nearest neighbors (KNN) is simple and easy to implement but may not be the best choice for this task. It requires storing all training data, which can be memory-intensive, especially for large datasets like Petfinder's.
* **Computational Efficiency**: The proposed solution may have high computational overhead due to feature extraction and clustering. This could lead to scalability issues, especially with millions of images in the dataset.
* **Interpretability**: The proposed solution lacks interpretability, which conflicts with the requirement for stakeholders, especially non-technical ones, to understand the system's decisions.

# Recommendation
Based on the critical analysis and stakeholders' wishes, the following recommendations are made:

1. **Feature Selection**:
Instead of solely relying on HOG features, a more comprehensive feature extraction method like convolutional neural networks (CNNs) should be considered. CNNs can automatically learn relevant features from images, which may better handle variations in imaging conditions.
2. **Classifier Selection**:
Instead of KNN, a more sophisticated classifier like a support vector machine (SVM) or a deep neural network (DNN) should be explored. These classifiers can provide better performance and scalability.
3. **Computational Efficiency**:
To address computational overhead, a trade-off between computational complexity and accuracy should be considered. This might involve using a smaller vocabulary size for visual words or optimizing the clustering algorithm.
4. **Involvement of Online Content Moderators**:
Develop features or functionalities that involve online content moderators in the system, ensuring they have a role in its operation post-deployment.
5. **Interpretability**:
Incorporate techniques for model interpretability, such as attention mechanisms or feature visualization, to ensure that the system's decisions are understandable to all stakeholders.

## Conclusion:

The proposed solution based on bags of visual words with HOG features and KNN classifier has limitations regarding handling imaging variations, computational efficiency, and interpretability. Therefore, it is recommended to explore more advanced techniques such as CNNs for feature extraction and SVMs or DNNs for classification, while also considering the involvement of online content moderators and ensuring interpretability for stakeholders.

In [ ]:
# convert your notebook to html (the file is created in the same directory as this notebook, on your Google Drive)
!jupyter nbconvert --to html 2CWK50-ML.ipynb